<a href="https://colab.research.google.com/github/tej10/deep-learning/blob/master/DL_rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io
path = get_file(
    'nietzsche.txt',
   origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
with io.open(path, encoding='utf-8') as f:
   text = f.read().lower()
print('corpus length:', len(text))

corpus length: 600893


In [3]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

total chars: 57


In [4]:
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 200285


In [5]:
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
y[i, char_indices[next_chars[i]]] = 1

Vectorization...


In [6]:
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

Build model...


In [0]:
optimizer = RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [0]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [9]:
def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)
    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=20,
          callbacks=[print_callback])

Epoch 1/20
200285/200285 [==============================] - 222s 1ms/step - loss: 1.9896e-05

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "ve now to cease being
"merely moral" men"
ve now to cease being
"merely moral" men333ë3b33]3
33333vg333b33bb333bl333333333bb3b33333w33wb3]333]333v3g333l3wl333bbbë33333333b3ë3333]
3bw3333u3333]g3v_33333.g33bb3l33w0b3gbb3333333vbbgg]3ebw333bb303w3b330ë3333w33b3g3
e33t3333033r33333333l3v333333i33w33
363w3b3r3w333333r3w33_333v3w333333vv33u3333033333333b
333]r3333333wbb333333v333v3333.bww3333333b333333w33b333333lvwb33v33333bu3b33333333l3333333333l33v33bgg3bb33wb]3333i333333333333333
----- diversity: 0.5
----- Generating with seed: "ve now to cease being
"merely moral" men"
ve now to cease being
"merely moral" menbbwtbuxw33a3ttr3vnuvv3lb3;3hée3o6,zbru]glvnbui3boz3_333;3uvun68
éwe33:b5b333-]eo]33.;jhwvw33),35wz"]lbwivlwr8vvjuiobb_bu3qnb-ggle-]33
=
3g
3z3urgaëë,t]ou3d3w);lë'gg33g]a3gr3ä_n33eu0irw3]ig'bh'tlgg-33v]eb0